In [1]:
from cloudvolume import CloudVolume
import numpy as np

In [2]:
vol_im = CloudVolume("file:///cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed/ch1_iso")
vol_axons = CloudVolume("file:///cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed/axons")

In [9]:
from cloudvolume.exceptions import SkeletonDecodeError
for i in range(66):
    try:
        verts = vol_axons.skeleton.get(i).vertices
    except SkeletonDecodeError:
        continue
    points = np.array(np.divide(verts, vol_axons.resolution))
    points = [list(c) for c in points]

    print(f"ID: {i} w/{len(points)} points")

    with open("/cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/points/map_axons/target_space/" + str(i) + ".txt", "w") as f:
        for point in points:
            f.write(f"({point[0]},{point[1]},{point[2]})")
            f.write("\n")

ID: 1 w/2626 points
ID: 4 w/2309 points
ID: 11 w/5938 points
ID: 12 w/16319 points
ID: 13 w/2235 points
ID: 14 w/536 points
ID: 20 w/26117 points
ID: 35 w/35547 points
ID: 40 w/488 points
ID: 45 w/688 points
ID: 48 w/15884 points
ID: 49 w/1399 points
ID: 51 w/5170 points
ID: 53 w/1712 points
ID: 59 w/5911 points
ID: 60 w/2049 points
ID: 65 w/2297 points


In [ ]:
from cloudreg.scripts.transform_points import NGLink
from cloudreg.scripts.visualization import create_viz_link_from_json

viz_link_init = "https://viz.neurodata.io/?json_url=https://json.neurodata.io/v1?NGStateID=7m7rEoVJdsNt1Q"

viz = NGLink(viz_link_init.split("json_url=")[-1])
ngl_json = viz._json
ngl_json['layers'].append(
    {
        "type": "annotation",
        "annotations": points,
        "name" : "test_points"
    }
)
viz_link = create_viz_link_from_json(ngl_json)
print(viz_link)

python -m cloudreg.scripts.transform_points --target_viz_link https://viz.neurodata.io/?json_url=https://json.neurodata.io/v1?NGStateID=x4nz2KzMfzinTQ --atlas_viz_link https://ara.viz.neurodata.io/?json_url=https://json.neurodata.io/v1?NGStateID=HvyNDGaPsd1wyg --affine_path /cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed_ch1_otsu_iso_registration/downloop_1_A.mat --velocity_path /cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed_ch1_otsu_iso_registration/downloop_1_v.mat --soma_path /cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/path_coords.txt

### Write new skeleton layer

### Read transformed points

In [20]:

from os import listdir
from os.path import isfile, join
import json

dir = '/cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/points/map_axons/atlas_space'
names = [f for f in listdir(dir) if isfile(join(dir, f))]
nums = [n.split('points')[-1][:-5] for n in names]
files = [join(dir,f) for f in names]

for num,file in zip(nums, files):
    skel_targ = vol_axons.skeleton.get(num)
    
    print(skel_targ)
    with open(file) as f:
        json_file = json.load(f)
        print(len(json_file))
    break


[Skeleton(segid=1, vertices=(shape=2626, float32), edges=(shape=2625, uint32), radius=(2626, float32), vertex_types=(2626, uint8), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])]
2626
